# Flux-Matching Weight Initialization

This notebook demonstrates the **iterative flux-matching** algorithm for initializing SOEN network weights.

---

## 🔊 NOISE CONFIGURATION: ENABLED (Default)

> **This tutorial runs with NOISE INJECTION (documented defaults).**
>
> | Parameter | Default | Description |
> |-----------|---------|-------------|
> | `phi` | **0.01** | Noise on input flux |
> | `s` | **0.005** | Noise on state |
> | `relative` | **False** | Absolute scaling |
>
> **To toggle noise on/off:** Use the `NOISE_ENABLED` variable in the next code cell.

---

## Why Flux-Matching?

The SOEN source function (RateArray/Heaviside) is periodic with **maximum response at φ_total = 0.5**:

$$\phi_{\text{total}} = \phi_{\text{exc}} + \phi_{\text{offset}}$$

For optimal network operation, we want each neuron's total input flux near this sweet spot. The flux-matching algorithm:

1. Runs forward passes with your training data
2. Observes actual mean states of each neuron
3. Adjusts weights so total incoming flux hits the target
4. Iterates until convergence

## Setup


In [ ]:
# Setup: Ensure soen_toolkit is importable
import sys
from pathlib import Path

# Add src directory to path if running from notebook location
notebook_dir = Path.cwd()
for parent in [notebook_dir] + list(notebook_dir.parents):
    candidate = parent / "src"
    if (candidate / "soen_toolkit").exists():
        sys.path.insert(0, str(candidate))
        break

import matplotlib.pyplot as plt
import numpy as np
import torch

# SOEN Toolkit imports
from soen_toolkit.core import SOENModelCore
from soen_toolkit.core.model_yaml import build_model_from_yaml
from soen_toolkit.utils.flux_matching_init import (
    FluxMatchingConfig,
    load_hdf5_batches,
    run_flux_matching_iterations,
)

# Set up plotting style
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11

print("Setup complete!")

In [ ]:
# ==============================================================================
# NOISE CONFIGURATION TOGGLE
# ==============================================================================
# Set NOISE_ENABLED = False to run with ideal conditions (no noise)
# Set NOISE_ENABLED = True for noise injection (default)

NOISE_ENABLED = True  # Toggle this to enable/disable noise

# Default noise parameters (documented defaults)
NOISE_DEFAULTS = {
    "phi": 0.01,           # Noise on input flux
    "s": 0.005,            # Noise on state
    "g": 0.0,              # Source function noise
    "bias_current": 0.0,   # Bias current noise
    "j": 0.0,              # Connection weight noise
    "relative": False,     # Absolute scaling
}

def set_model_noise(model, enabled=True, noise_values=None):
    """
    Toggle noise injection on/off for a SOEN model.
    
    Args:
        model: SOENModelCore instance
        enabled: If True, apply noise; if False, set all noise to 0
        noise_values: Dict of noise parameters (uses NOISE_DEFAULTS if None)
    
    Returns:
        model: The modified model (for chaining)
    """
    from soen_toolkit.core.configs import NoiseConfig
    
    if noise_values is None:
        noise_values = NOISE_DEFAULTS
    
    # Update layer noise configurations
    for cfg in model.layers_config:
        if enabled:
            cfg.noise = NoiseConfig(
                phi=noise_values.get("phi", 0.01),
                s=noise_values.get("s", 0.005),
                g=noise_values.get("g", 0.0),
                bias_current=noise_values.get("bias_current", 0.0),
                j=noise_values.get("j", 0.0),
                relative=noise_values.get("relative", False),
                extras=getattr(cfg.noise, "extras", {}),
            )
        else:
            cfg.noise = NoiseConfig(
                phi=0.0, s=0.0, g=0.0, bias_current=0.0, j=0.0,
                relative=False,
                extras=getattr(cfg.noise, "extras", {}),
            )
    
    # Update connection noise configurations
    for conn_cfg in model.connections_config:
        if enabled:
            conn_cfg.noise = NoiseConfig(
                phi=0.0, g=0.0, s=0.0, bias_current=0.0,
                j=noise_values.get("j", 0.0),
                relative=noise_values.get("relative", False),
                extras={},
            )
        else:
            conn_cfg.noise = NoiseConfig(
                phi=0.0, g=0.0, s=0.0, bias_current=0.0, j=0.0,
                relative=False, extras={},
            )
    
    status = "ENABLED" if enabled else "DISABLED"
    print(f"✓ Noise injection {status}")
    if enabled:
        print(f"  phi={noise_values['phi']}, s={noise_values['s']}, "
              f"relative={noise_values['relative']}")
    
    return model

print(f"Noise injection: {'ENABLED' if NOISE_ENABLED else 'DISABLED'}")
if NOISE_ENABLED:
    print(f"  Default values: phi={NOISE_DEFAULTS['phi']}, s={NOISE_DEFAULTS['s']}, "
          f"relative={NOISE_DEFAULTS['relative']}")

---
## 1. Configuration

Set your paths and algorithm parameters below. Modify these to use your own model and data.


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# USER CONFIGURATION - Modify these paths and settings
# ═══════════════════════════════════════════════════════════════════════════════

# Path to your model YAML file
MODEL_YAML_PATH = "path/to/your/model.yaml"


# Path to your HDF5 dataset
DATA_HDF5_PATH = "path/to/your/data.hdf5"

# Dataset split to use ('train', 'val', 'test', or 'all_data')
DATA_SPLIT = "train"

# Data scaling - normalize input data to this range
# Set both to None to skip scaling
DATA_SCALE_MIN = 0.0  # Target minimum value
DATA_SCALE_MAX = 1.0  # Target maximum value

# Path to save the initialized model
OUTPUT_MODEL_PATH = "flux_matched_model.soen"

# ═══════════════════════════════════════════════════════════════════════════════
# ALGORITHM SETTINGS
# ═══════════════════════════════════════════════════════════════════════════════

# Target TOTAL flux (phi_total = phi_exc + phi_offset)
# The algorithm reads each node's phi_offset and computes the required phi_exc.
# The optimal operating point is phi_total = 0.5 (maximum source function response).

# Option 1: Single target for all nodes
PHI_TOTAL_TARGET = 0.5

# Option 2: Symmetry breaking with a range of targets
# When both min and max are specified, targets are uniformly distributed
# across dendrites within each layer (from min at node 0 to max at node N-1).
# This breaks symmetry deterministically without adding noise.
# Set both to None to use the single PHI_TOTAL_TARGET value instead.
PHI_TOTAL_TARGET_MIN = 0.45  # Lower bound of target phi_total
PHI_TOTAL_TARGET_MAX = 0.55  # Upper bound of target phi_total

# Number of flux-matching iterations
NUM_ITERATIONS = 100

# Batch size for forward passes
BATCH_SIZE = 64

# Number of batches to use (None = use all data, or set a number for faster runs)
NUM_BATCHES = 20  # Set to None to use all data

# Minimum state clamp (prevents division by zero)
MIN_STATE_CLAMP = 0.01

# Step size for weight updates (0=no update, 1=full update, 0.5=smooth updates)
ALPHA = 0.1

# Weight update mode: "connection_wise" or "node_wise"
#   - connection_wise: Each J_ij is computed from individual source state s_j (more precise)
#   - node_wise: All J for a destination use averaged upstream states (simpler)
WEIGHT_UPDATE_MODE = "connection_wise"

# Device ('cpu' or 'cuda')
DEVICE = "cpu"

print("Configuration set!")
print(f"  Model: {MODEL_YAML_PATH}")
print(f"  Data: {DATA_HDF5_PATH}")
if PHI_TOTAL_TARGET_MIN is not None and PHI_TOTAL_TARGET_MAX is not None:
    print(f"  Target phi_total range: {PHI_TOTAL_TARGET_MIN} to {PHI_TOTAL_TARGET_MAX} (symmetry breaking)")
else:
    print(f"  Target phi_total: {PHI_TOTAL_TARGET}")
print(f"  Iterations: {NUM_ITERATIONS}")
print(f"  Mode: {WEIGHT_UPDATE_MODE}")
print(f"  Alpha: {ALPHA}")


### Validate Paths and Load Model


In [ ]:
# Check if files exist
model_path = Path(MODEL_YAML_PATH)
data_path = Path(DATA_HDF5_PATH)

if not model_path.exists():
    print(f"Model file not found: {model_path}")
    print("Please update MODEL_YAML_PATH in the configuration cell above.")
else:
    print(f"Model file found: {model_path}")

if not data_path.exists():
    print(f"Data file not found: {data_path}")
    print("Please update DATA_HDF5_PATH in the configuration cell above.")
else:
    print(f"Data file found: {data_path}")

    # Show data info
    import h5py
    with h5py.File(data_path, 'r') as f:
        print("\nDataset contents:")
        for key in f.keys():
            if hasattr(f[key], 'shape'):
                print(f"  {key}: {f[key].shape}")
            else:
                print(f"  {key}/ (group)")


---
## 2. Load Model and Data


In [ ]:
# Load the model (supports both .yaml and .soen formats)
print("Loading model...")
if model_path.suffix.lower() in ['.soen', '.pth']:
    model = SOENModelCore.load(str(model_path))
else:
    model = build_model_from_yaml(model_path)
model = model.to(DEVICE)

print("\nModel Architecture:")
print(f"  Layers: {len(model.layers_config)}")
for cfg in model.layers_config:
    dim = cfg.params.get('dim', '?')
    print(f"    Layer {cfg.layer_id}: {cfg.layer_type} (dim={dim})")

print(f"\n  Connections: {len(model.connections_config)}")
for conn in model.connections_config:
    print(f"    {conn.from_layer} -> {conn.to_layer} ({conn.connection_type})")


In [ ]:
# Load data batches
print(f"Loading data from {DATA_SPLIT} split...")
data_batches = load_hdf5_batches(
    data_path,
    split=DATA_SPLIT,
    batch_size=BATCH_SIZE,
    num_batches=NUM_BATCHES,
    device=DEVICE,
)

# Scale data if requested
if DATA_SCALE_MIN is not None and DATA_SCALE_MAX is not None:
    # Find global min/max across all batches
    all_data = torch.cat(data_batches, dim=0)
    data_min = all_data.min()
    data_max = all_data.max()
    print(f"\nOriginal data range: [{data_min:.4f}, {data_max:.4f}]")

    # Scale to target range
    scale = (DATA_SCALE_MAX - DATA_SCALE_MIN) / (data_max - data_min + 1e-8)
    data_batches = [
        (batch - data_min) * scale + DATA_SCALE_MIN
        for batch in data_batches
    ]
    print(f"Scaled to range: [{DATA_SCALE_MIN}, {DATA_SCALE_MAX}]")

print(f"\nLoaded {len(data_batches)} batches")
if data_batches:
    sample = data_batches[0]
    print(f"  Batch shape: {sample.shape}")
    print(f"  Data range: [{sample.min():.4f}, {sample.max():.4f}]")
    print("  (batch_size, sequence_length, input_dim)")


---
## 3. Run Flux-Matching Algorithm

Now let's run the iterative flux-matching to optimize the weights.


In [ ]:
# Configure the algorithm
config = FluxMatchingConfig(
    phi_total_target=PHI_TOTAL_TARGET,
    phi_total_target_min=PHI_TOTAL_TARGET_MIN,  # Set to None to disable symmetry breaking
    phi_total_target_max=PHI_TOTAL_TARGET_MAX,  # Set to None to disable symmetry breaking
    num_iterations=NUM_ITERATIONS,
    batch_size=BATCH_SIZE,
    num_batches=NUM_BATCHES,
    min_state_clamp=MIN_STATE_CLAMP,
    alpha=ALPHA,
    weight_update_mode=WEIGHT_UPDATE_MODE,
    verbose=True,
)

print("Starting Flux-Matching Optimization")
print("=" * 60)

# Run the algorithm
result = run_flux_matching_iterations(model, data_batches, config)

print("\n" + "=" * 60)
if result.converged:
    print("CONVERGED to target phi_total!")
else:
    print("Did not fully converge. Consider more iterations or smaller alpha.")


---
## 4. Visualize Convergence


In [ ]:
def plot_convergence(result, target_flux=None, target_min=None, target_max=None):
    """Plot flux convergence over iterations.
    
    Args:
        result: FluxMatchingResult from run_flux_matching_iterations
        target_flux: Single target flux value (used if min/max not provided)
        target_min: Lower bound of target flux range
        target_max: Upper bound of target flux range
    """

    # Extract layers with flux data
    layers_with_flux = set()
    for stats in result.iteration_stats:
        layers_with_flux.update(stats["flux_per_layer"].keys())

    fig, axes = plt.subplots(1, 3, figsize=(16, 5))

    # Plot 1: Flux Convergence
    ax1 = axes[0]
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(layers_with_flux)))

    for i, layer_id in enumerate(sorted(layers_with_flux)):
        means = []
        stds = []
        for stats in result.iteration_stats:
            if layer_id in stats["flux_per_layer"]:
                means.append(stats["flux_per_layer"][layer_id]["mean"])
                stds.append(stats["flux_per_layer"][layer_id]["std"])

        iterations = list(range(1, len(means) + 1))
        ax1.fill_between(iterations,
                         [m - s for m, s in zip(means, stds, strict=False)],
                         [m + s for m, s in zip(means, stds, strict=False)],
                         alpha=0.2, color=colors[i])
        ax1.plot(iterations, means, 'o-', color=colors[i],
                 label=f'Layer {layer_id}', linewidth=2, markersize=6)

    # Draw target as either a range or single line
    if target_min is not None and target_max is not None:
        ax1.axhspan(target_min, target_max, color='red', alpha=0.15, label=f'Target range ({target_min}-{target_max})')
        mid = (target_min + target_max) / 2
        ax1.axhline(y=mid, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label=f'Midpoint ({mid:.2f})')
    elif target_flux is not None:
        ax1.axhline(y=target_flux, color='red', linestyle='--', linewidth=2, label=f'Target ({target_flux})')

    ax1.set_xlabel('Iteration', fontsize=12)
    ax1.set_ylabel('Mean Flux per Node', fontsize=12)
    ax1.set_title('Flux Convergence', fontsize=14, fontweight='bold')
    ax1.legend(loc='best')
    ax1.set_xlim(0.5, len(result.iteration_stats) + 0.5)
    ax1.grid(True, alpha=0.3)

    # Plot 2: Mean State Evolution
    ax2 = axes[1]
    all_layers = set()
    for stats in result.iteration_stats:
        all_layers.update(stats["mean_states"].keys())

    colors2 = plt.cm.plasma(np.linspace(0.2, 0.8, len(all_layers)))

    for i, layer_id in enumerate(sorted(all_layers)):
        means = [stats["mean_states"].get(layer_id, float('nan')) for stats in result.iteration_stats]
        iterations = list(range(1, len(means) + 1))
        ax2.plot(iterations, means, 's-', color=colors2[i], label=f'Layer {layer_id}', linewidth=2, markersize=6)

    ax2.set_xlabel('Iteration', fontsize=12)
    ax2.set_ylabel('Mean State', fontsize=12)
    ax2.set_title('Mean State per Layer', fontsize=14, fontweight='bold')
    ax2.legend(loc='best')
    ax2.set_xlim(0.5, len(result.iteration_stats) + 0.5)
    ax2.grid(True, alpha=0.3)

    # Plot 3: Error Reduction
    ax3 = axes[2]

    for i, layer_id in enumerate(sorted(layers_with_flux)):
        errors = [stats["flux_error_per_layer"].get(layer_id, float('nan')) for stats in result.iteration_stats]
        iterations = list(range(1, len(errors) + 1))
        ax3.semilogy(iterations, errors, 'o-', color=colors[i], label=f'Layer {layer_id}', linewidth=2, markersize=6)

    ax3.axhline(y=0.02, color='green', linestyle=':', linewidth=2, label='Convergence threshold')
    ax3.set_xlabel('Iteration', fontsize=12)
    ax3.set_ylabel('Flux Error (log scale)', fontsize=12)
    ax3.set_title('Error Reduction', fontsize=14, fontweight='bold')
    ax3.legend(loc='best')
    ax3.set_xlim(0.5, len(result.iteration_stats) + 0.5)
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    return fig

# Generate the plot
fig = plot_convergence(
    result,
    target_flux=PHI_TOTAL_TARGET,
    target_min=PHI_TOTAL_TARGET_MIN,
    target_max=PHI_TOTAL_TARGET_MAX,
)
plt.show()

# Save the figure
fig.savefig('flux_matching_convergence.png', dpi=150, bbox_inches='tight')
print("\nPlot saved to: flux_matching_convergence.png")


---
## 5. Final Summary


In [ ]:
# Display final statistics
final_stats = result.iteration_stats[-1]

print("=" * 70)
print("FINAL STATE SUMMARY")
print("=" * 70)

# Determine target info for display
using_range = PHI_TOTAL_TARGET_MIN is not None and PHI_TOTAL_TARGET_MAX is not None
if using_range:
    target_mid = (PHI_TOTAL_TARGET_MIN + PHI_TOTAL_TARGET_MAX) / 2
    print(f"\nTarget phi_total range: {PHI_TOTAL_TARGET_MIN} to {PHI_TOTAL_TARGET_MAX} (symmetry breaking)")
    print(f"Expected mean phi_total: {target_mid:.3f}")
else:
    target_mid = PHI_TOTAL_TARGET
    print(f"\nTarget phi_total: {PHI_TOTAL_TARGET}")

print(f"Iterations:  {NUM_ITERATIONS}")
print(f"Converged:   {'Yes' if result.converged else 'No'}")

print("\n" + "-" * 80)
if using_range:
    print(f"{'Layer':<8} {'Mean State':<12} {'phi_total [min, max]':<22} {'Mean':<10} {'vs Target':<15}")
else:
    print(f"{'Layer':<8} {'Mean State':<12} {'phi_total (mean +/- std)':<27} {'Error %':<10}")
print("-" * 80)

for layer_id in sorted(final_stats["mean_states"].keys()):
    mean_s = final_stats["mean_states"][layer_id]
    if layer_id in final_stats["flux_per_layer"]:
        flux_info = final_stats["flux_per_layer"][layer_id]
        error = final_stats["flux_error_per_layer"][layer_id]
        error_pct = (error / target_mid) * 100
        if using_range:
            range_str = f"[{flux_info['min']:.3f}, {flux_info['max']:.3f}]"
            target_range = f"[{PHI_TOTAL_TARGET_MIN}, {PHI_TOTAL_TARGET_MAX}]"
            print(f"{layer_id:<8} {mean_s:<12.4f} {range_str:<22} {flux_info['mean']:<10.4f} {target_range:<15}")
        else:
            flux_str = f"{flux_info['mean']:.4f} +/- {flux_info['std']:.4f}"
            print(f"{layer_id:<8} {mean_s:<12.4f} {flux_str:<27} {error_pct:>6.2f}%")
    else:
        print(f"{layer_id:<8} {mean_s:<12.4f} {'(input layer)':<27}")

print("-" * 80)


---
## 6. Optional: Impose an Inhibitory Fraction (Preserve Target Flux)

Flux-matching can yield mostly-positive weights (especially when mean upstream states are mostly positive). If you want (for example) **20% inhibitory** synapses per destination node (per incoming edge), you can apply a post-processing step that:

- flips a random fraction of active synapses to negative
- rescales the remaining active synapses so each destination node still matches its target external flux

This uses the same `mean_states` estimate that flux matching used.



In [ ]:
from soen_toolkit.utils.flux_matching_init import apply_inhibitory_fraction_preserving_flux

# Apply after flux matching, before saving (optional)
# Uses the final mean_states from the last iteration
apply_inhibitory_fraction_preserving_flux(
    model,
    result.final_mean_states,
    config=config,
    inhibitory_fraction=0.2,
    seed=0,
)

print("Applied inhibitory fraction post-processing.")



---
## 6. Save the Initialized Model

Save the model with optimized weights using `.soen` format (binary format that preserves weights, masks, and configuration).


In [ ]:
# Save the model as .soen (includes weights, masks, and config)
output_path = Path(OUTPUT_MODEL_PATH)

print(f"Saving initialized model to: {output_path}")
model.save(str(output_path))

print("\nModel saved successfully!")
print("\nTo load this model later:")
print("  from soen_toolkit.core import SOENModelCore")
print(f"  model = SOENModelCore.load('{output_path}')")


---
## 7. Weight Distribution (Optional)

Visualize the final weight distributions.


In [ ]:
# Plot weight distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

conn_keys = list(model.connections.keys())

for i, key in enumerate(conn_keys[:6]):  # Show up to 6 connections
    ax = axes[i]
    weights = model.connections[key].detach().cpu().numpy().flatten()

    # Filter by mask if available
    mask = model.connection_masks.get(key)
    if mask is not None:
        mask_np = mask.cpu().numpy().flatten()
        weights = weights[mask_np > 0]  # Only show active weights

    ax.hist(weights, bins=50, alpha=0.7, color='steelblue', edgecolor='white')
    ax.axvline(x=weights.mean(), color='red', linestyle='--', linewidth=2,
               label=f'Mean: {weights.mean():.4f}')
    ax.set_title(key, fontsize=11, fontweight='bold')
    ax.set_xlabel('Weight Value')
    ax.set_ylabel('Count')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

# Hide unused subplots
for i in range(len(conn_keys), 6):
    axes[i].set_visible(False)

plt.suptitle('Weight Distributions After Flux-Matching', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()


---
## Summary

You have successfully:

1. **Loaded** your model and training data
2. **Ran** the iterative flux-matching algorithm  
3. **Visualized** the convergence process
4. **Saved** the optimized model (`.soen` format with weights)

### Parameter Reference

| Parameter | Description | Default |
|-----------|-------------|---------|
| `phi_total_target` | Target total flux (phi_exc + phi_offset). 0.5 is the optimal operating point. | 0.5 |
| `phi_total_target_min` | Lower bound of target phi_total range (for symmetry breaking). Set to None to use single target. | None |
| `phi_total_target_max` | Upper bound of target phi_total range (for symmetry breaking). Set to None to use single target. | None |
| `num_iterations` | Number of optimization iterations | 5 |
| `batch_size` | Batch size for forward passes | 32 |
| `num_batches` | Number of batches to use (None = all data) | None |
| `min_state_clamp` | Minimum state value to prevent division by zero | 0.01 |
| `alpha` | Step size for weight updates (0=none, 1=full, 0.5=smooth) | 1.0 |
| `weight_update_mode` | `"connection_wise"` or `"node_wise"` (see below) | `"connection_wise"` |

### Per-Node phi_offset Handling

The algorithm automatically reads each node's `phi_offset` from the layer and computes the required external flux:

```
phi_exc_target[i] = phi_total_target[i] - phi_offset[i]
```

This means you specify the actual operating point (`phi_total`) and the algorithm figures out what weights are needed to achieve it, regardless of how phi_offset varies across nodes.

### Symmetry Breaking

When `phi_total_target_min` and `phi_total_target_max` are both specified, target phi_total values are **uniformly distributed** across dendrites within each layer:
- Node 0 gets target = `phi_total_target_min`
- Node N-1 gets target = `phi_total_target_max`
- Intermediate nodes are linearly interpolated

This breaks symmetry deterministically without adding noise. Example:
```python
config = FluxMatchingConfig(
    phi_total_target_min=0.45,
    phi_total_target_max=0.55,
    ...
)
```

For a layer with 100 nodes, targets would range from 0.45 (node 0) to 0.55 (node 99).

### Weight Update Modes

**Connection-wise** (recommended): Computes individual J_ij for each connection:
```
J_ij = phi_exc_target[i] / (num_sources * fan_in_i * s_j)
```
Each weight uses the actual state s_j of its specific source neuron.

**Node-wise**: Computes uniform J for each destination node:
```
J_i = phi_exc_target[i] / (num_sources * fan_in_i * mean_upstream_i)
```
Uses averaged upstream states (simpler but less precise).

### How It Works

1. **Forward pass**: Run input data through the network
2. **State collection**: Record mean neuron states across time and batches  
3. **Flux calculation**: Compute current phi_total for each neuron (phi_exc + phi_offset)
4. **Weight adjustment**: Compute target phi_exc accounting for each node's phi_offset, then blend weights toward target
5. **Repeat**: Iterate until phi_total converges to target

### Next Steps

- Use the initialized model for training
- Compare training convergence: random init vs flux-matched init
- Experiment with different `phi_total_target` values or ranges
- Try smaller `alpha` (e.g., 0.5) if oscillations occur


---
## Summary (Updated Notes)

This tutorial now supports two extra concepts that aren’t reflected in the older Summary section above:

- **Target allocation across incoming edges**: if a destination layer has multiple incoming weight matrices, the per-node target external flux is allocated across those incoming edges using `flux_allocation_mode` (`"equal_per_incoming_connection"` or `"proportional_to_fan_in"`).
- **Inhibitory fraction post-step**: you can impose a chosen fraction of inhibitory (negative) synapses after flux matching while preserving each node’s target external flux.

### Correct per-edge weight formulas

Within a single incoming edge:

**Connection-wise**:

```
phi_exc_target_for_edge[i] = allocation(phi_exc_target[i])
J_edge[i,j] = phi_exc_target_for_edge[i] / (fan_in_edge[i] * s_j)
```

**Node-wise**:

```
phi_exc_target_for_edge[i] = allocation(phi_exc_target[i])
J_edge[i,:] = phi_exc_target_for_edge[i] / (fan_in_edge[i] * mean_upstream_edge[i])
```

### Inhibitory fraction (preserve flux)

After `run_flux_matching_iterations(...)`:

```
apply_inhibitory_fraction_preserving_flux(
    model,
    result.final_mean_states,
    config=config,
    inhibitory_fraction=0.2,
    seed=0,
)
```

